In [1]:
from yugiquery import *
header('Speed Duel')

<div align='center'>
    <br>
    <!-- Pre Style needed to fix HTML formatting -->
    <pre style="line-height: var(--jp-code-line-height); font-family: var(--jp-code-font-family)">
    ██    ██ ██    ██  ██████  ██  ██████  ██    ██ ███████ ██████  ██    ██ 
     ██  ██  ██    ██ ██       ██ ██    ██ ██    ██ ██      ██   ██  ██  ██  
      ████   ██    ██ ██   ███ ██ ██    ██ ██    ██ █████   ██████    ████   
       ██    ██    ██ ██    ██ ██ ██ ▄▄ ██ ██    ██ ██      ██   ██    ██    
       ██     ██████   ██████  ██  ██████   ██████  ███████ ██   ██    ██    
                                      ▀▀                                     
    </pre>
</div>
<div align='right'>
    Speed Duel - Execution started 16/02/2023 19:02 UTC <br>
    By <b>Guilherme Ruiz</b>
</div>

---

# Data aquisition

## Fetch online data

In [2]:
# Timestamp
timestamp = pd.Timestamp.now()

In [3]:
# Fetch skill cards
skill_df = fetch_skill()

# Fetch deck cards
speed_df = fetch_speed()

103 results



893 results



## Merge data

In [4]:
full_df = pd.concat([speed_df,skill_df], ignore_index=True, axis=0).sort_values('Name').reset_index(drop=True)
print('Data merged')

Data merged


## Save data

In [5]:
full_df.to_csv(f'../data/all_speed_{timestamp.isoformat(timespec="minutes")}.csv', index=False)
print('Data saved')

Data saved


# Check changes

## Load previous data

In [6]:
# Get list of files
files_list = sorted(glob.glob('../data/all_speed_*.csv'), key=os.path.getctime, reverse=True)
# Get second newest file if exist
if len(files_list)>1:
    # Load csv avoiding converting "NA" to NaN
    previous_df = pd.read_csv(files_list[1], dtype=object, keep_default_na=False, na_values=[''])
    # Correct tuples
    previous_df['Secondary type'] = previous_df['Secondary type'].dropna().apply(literal_eval)
    previous_df['Effect type'] = previous_df['Effect type'].dropna().apply(literal_eval)
    previous_df['Archseries'] = previous_df['Archseries'].dropna().apply(literal_eval)
    previous_df['Artwork'] = previous_df['Artwork'].dropna().apply(literal_eval)
    # previous_df['Errata'] = previous_df['Errata'].dropna().apply(literal_eval)
    # Force dtypes to match current df
    previous_df = previous_df.astype(full_df[previous_df.columns].dtypes.to_dict())
    previous_ts = pd.to_datetime(os.path.basename(files_list[1]).split('_')[-1].rstrip('.csv'))
    print('File loaded')
else:
    previous_df = None
    print('No older files')

File loaded


## Generate changelog

In [7]:
if previous_df is None:
    print('Skipped')
else:
    changelog = generate_changelog(previous_df, full_df, col = 'Name')
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(f'../data/speed_changelog_{timestamp.isoformat(timespec="minutes")}_{previous_ts.isoformat(timespec="minutes")}.csv', index = True)
        print('Changelog saved')

No changes


# Data visualization

In [8]:
full_df

,Name,Password,Card type,Property,Archseries,Attribute,Primary type,Secondary type,Monster type,Effect type,Level/Rank,DEF,ATK,TCG status,OCG status,TCG Speed Duel status,Page name,Page URL,Artwork,Modification date,Character
0,7 Completed,86198326,Spell Card,Equip Spell Card,NaN,NaN,NaN,NaN,NaN,"(Condition, Continuous-like Effect)",NaN,NaN,NaN,Unlimited,Unlimited,Unlimited,7 Completed,https://yugipedia.com/wiki/7_Completed,NaN,2021-10-31 17:18:50,NaN
1,A Cat of Ill Omen,24140059,Monster Card,NaN,NaN,DARK,Effect Monster,"(Flip monster,)",Beast,"(Flip effect,)",2,300,500,Unlimited,Unlimited,Unlimited,A Cat of Ill Omen,https://yugipedia.com/wiki/A_Cat_of_Ill_Omen,NaN,2022-04-11 17:39:44,NaN
2,A Hero Emerges,21597117,Trap Card,Normal Trap Card,NaN,NaN,NaN,NaN,NaN,"(Activation condition, Effect)",NaN,NaN,NaN,Unlimited,Unlimited,Unlimited,A Hero Emerges,https://yugipedia.com/wiki/A_Hero_Emerges,NaN,2022-03-21 20:21:26,NaN
3,A Hero Lives,08949584,Spell Card,Normal Spell Card,NaN,NaN,NaN,NaN,NaN,"(Activation condition, Cost, Effect)",NaN,NaN,NaN,Unlimited,Unlimited,Unlimited,A Hero Lives,https://yugipedia.com/wiki/A_Hero_Lives,NaN,2022-11-02 12:48:15,NaN
4,A Legendary Ocean,00295517,Spell Card,Field Spell Card,"(Umi (series),)",NaN,NaN,NaN,NaN,"(Condition, Continuous-like Effect)",NaN,NaN,NaN,Unlimited,Unlimited,Unlimited,A Legendary Ocean,https://yugipedia.com/wiki/A_Legendary_Ocean,NaN,2022-11-04 14:25:19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,Zoma the Spirit,79852326,Trap Card,Continuous Trap Card,"(Trap Monster,)",NaN,NaN,NaN,NaN,"(Effect, Trigger Effect)",NaN,NaN,NaN,Unlimited,Unlimited,Unlimited,Zoma the Spirit,https://yugipedia.com/wiki/Zoma_the_Spirit,NaN,2022-09-02 12:21:41,NaN
992,Zombie Master,NaN,Skill Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Illegal,NaN,Legal,Zombie Master (Skill Card),https://yugipedia.com/wiki/Zombie_Master_(Skil...,NaN,2022-12-27 15:38:32,Bonz
993,Zombie Tiger,47693640,Monster Card,NaN,NaN,EARTH,Effect Monster,"(Union monster,)",Zombie,"(Condition, Continuous-like Effect, Ignition E...",3,1600,1400,Unlimited,Unlimited,Unlimited,Zombie Tiger,https://yugipedia.com/wiki/Zombie_Tiger,NaN,2022-07-24 09:37:57,NaN
994,Zombina,81616639,Monster Card,NaN,NaN,EARTH,Effect Monster,NaN,Zombie,"(Trigger Effect,)",4,1500,1400,Unlimited,Unlimited,Unlimited,Zombina,https://yugipedia.com/wiki/Zombina,NaN,2022-05-24 15:59:37,NaN


## Property

In [9]:
full_df['Property'].nunique()

9

In [10]:
full_df['Property'].value_counts()

Normal Spell Card        100
Normal Trap Card          98
Equip Spell Card          40
Field Spell Card          30
Continuous Trap Card      30
Continuous Spell Card     29
Quick-Play Spell Card     19
Ritual Spell Card         11
Counter Trap Card         11
Name: Property, dtype: int64

## Archseries

In [11]:
full_df['Archseries'].explode().nunique()

151

In [12]:
full_df['Archseries'].explode().value_counts()

HERO                        34
Cyber                       27
roid                        23
Elemental HERO              22
Crystal                     19
                            ..
Foolish                      1
Flame Swordsman (series)     1
Fishborg                     1
Fire Formation               1
Worm                         1
Name: Archseries, Length: 151, dtype: int64

## TCG Speed Duel status

In [13]:
full_df['TCG Speed Duel status'].nunique()

3

In [14]:
full_df['TCG Speed Duel status'].value_counts()

Unlimited           884
Legal               103
Not yet released      9
Name: TCG Speed Duel status, dtype: int64

## TCG status

In [15]:
full_df['TCG status'].nunique()

4

In [16]:
full_df['TCG status'].value_counts()

Unlimited       889
Illegal         103
Semi-Limited      2
Limited           2
Name: TCG status, dtype: int64

## Character

In [17]:
skill_df['Character'].nunique()

36

In [18]:
skill_df['Character'].value_counts()

Yami Yugi              10
Seto Kaiba              6
Joey Wheeler            5
Bonz                    5
Maximillion Pegasus     5
Ishizu Ishtar           5
Jaden Yuki              4
Rex Raptor              4
Syrus Truesdale         4
Bandit Keith            4
Mai Valentine           4
Mako Tsunami            4
Paradox Brothers        3
Bastion Misawa          3
Weevil Underwood        3
Yami Marik              3
Aster Phoenix           2
Espa Roba               2
Yami Bakura             2
Vellian Crowler         2
Odion                   2
Chazz Princeton         2
Zane Truesdale          2
Alexis Rhodes           2
Jesse Anderson          2
Axel Brodie             2
Christine               1
David                   1
Tyranno Hassleberry     1
Yubel                   1
Chumley Huffington      1
Andrew                  1
Arkana                  1
Téa Gardner             1
Emma (Duel Links)       1
Lumis and Umbra         1
Name: Character, dtype: int64

# Epilogue

In [19]:
footer()

<div align='right'>
    Execution completed 16/02/2023 19:03 UTC
</div>

## HTML export

In [20]:
# Save notebook on disck before generating HTML report
save_notebook()

Notebook saved to disk


In [21]:
! jupyter nbconvert Speed.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

[NbConvertApp] Converting notebook Speed.ipynb to HTML


[NbConvertApp] Writing 599574 bytes to ../Speed.html


## Git

In [ ]:
! git add "../*[Ss]peed*"

In [ ]:
! git commit -m {"'Speed duel update-" + timestamp.isoformat() + "'"}